In [1]:
import sympy as sy
import numpy as np
import casadi as ca
import pathlib
from SE3Lie import *
import numpy as np
from sympy import *
from pyecca.lie import so3, r3, se3
from pyecca.lie.util import DirectProduct
from pyecca.lie.so3 import Quat, Dcm, Euler, Mrp
from pyecca.lie.r3 import R3
from pyecca.lie.se3 import SE3
from pyecca.test import test_lie
import casadi as ca
from SE3Lie import se3, SE3, se3_diff_correction_inv, dot_plot_draw
from casadi.tools.graph import graph, dotdraw, dotsave
import os
import pydot
import math
from Sympy_To_Casadi import SympyToCasadi


Sympy for U_inv Matrix for Reference

In [2]:
x,y,z,theta_1,theta_2,theta_3 = symbols('x,y,z,theta_1,theta_2,theta_3',real=True)
theta = symbols('theta', real = True)
ad = sy.Matrix( [[0, -theta_3, theta_2, 0, -z, y], [theta_3, 0, -theta_1, z,0, -x], [-theta_2, theta_1,0,-y, x, 0], [0,0,0,0,-theta_3,theta_2],[0,0,0,theta_3,0,-theta_1],[0,0,0,-theta_2,theta_1,0]])
I = eye(6)
A, B = symbols('A, B')
f_A = sy.Function('A_1')
f_B = sy.Function('B_1')
UInv = -1*(I + A*ad + B*ad**2)
UInv = UInv.subs(A, f_A(theta)).subs(B,f_B(theta))
UInv #separate into Odd and Even terms
ad

Matrix([
[       0, -theta_3,  theta_2,        0,       -z,        y],
[ theta_3,        0, -theta_1,        z,        0,       -x],
[-theta_2,  theta_1,        0,       -y,        x,        0],
[       0,        0,        0,        0, -theta_3,  theta_2],
[       0,        0,        0,  theta_3,        0, -theta_1],
[       0,        0,        0, -theta_2,  theta_1,        0]])

In [3]:
UInv

Matrix([
[      -(-theta_2**2 - theta_3**2)*B_1(theta) - 1, -theta_1*theta_2*B_1(theta) + theta_3*A_1(theta), -theta_1*theta_3*B_1(theta) - theta_2*A_1(theta),           -(-2*theta_2*y - 2*theta_3*z)*B_1(theta),  z*A_1(theta) - (theta_1*y + theta_2*x)*B_1(theta), -y*A_1(theta) - (theta_1*z + theta_3*x)*B_1(theta)],
[-theta_1*theta_2*B_1(theta) - theta_3*A_1(theta),       -(-theta_1**2 - theta_3**2)*B_1(theta) - 1,  theta_1*A_1(theta) - theta_2*theta_3*B_1(theta), -z*A_1(theta) - (theta_1*y + theta_2*x)*B_1(theta),           -(-2*theta_1*x - 2*theta_3*z)*B_1(theta),  x*A_1(theta) - (theta_2*z + theta_3*y)*B_1(theta)],
[-theta_1*theta_3*B_1(theta) + theta_2*A_1(theta), -theta_1*A_1(theta) - theta_2*theta_3*B_1(theta),       -(-theta_1**2 - theta_2**2)*B_1(theta) - 1,  y*A_1(theta) - (theta_1*z + theta_3*x)*B_1(theta), -x*A_1(theta) - (theta_2*z + theta_3*y)*B_1(theta),           -(-2*theta_1*x - 2*theta_2*y)*B_1(theta)],
[                                               0,                 

In [4]:
f_replacements = {}
A = ca.SX.sym('A')


C1 = (1-ca.cos((A)))/(ca.mpower(A,2))
C2 = (A-ca.sin(A))/(ca.mpower(A,3))

f_replacements['A_1'] = ca.Function('theta',[A], [
    ca.if_else(type(A) == ca.casadi.SX, (C1), 
    ca.if_else(ca.fabs(A) < 1e-5, 0.5 - A ** 2 / 24 + A ** 4 / 720, C1 )) #fix ca.taylor expr
    ])
f_replacements['B_1'] = ca.Function('theta',[A], [
    ca.if_else(type(A) == ca.casadi.SX, C2,
    ca.if_else(ca.fabs(A) < 1e-5, 1/6 - A**2 /120 + A**4/5040, C2))  #fix ca.taylor expr
    ])
    
f_replacements

{'A_1': Function(theta:(i0)->(o0) SXFunction),
 'B_1': Function(theta:(i0)->(o0) SXFunction)}

U Inv matrix in casadi form and dot draw plot


parser = SympyToCasadi()
UInv_ca = parser.parse(UInv, cse=True)
UInv_ca

In [5]:
parser = SympyToCasadi()
UInv_ca = parser.parse(UInv, f_replacements, cse=True)
UInv_ca

SX(@1=-1, @2=1, @3=((@2-cos(theta))/sq(theta)), @4=sq(theta_2), @5=sq(theta_3), @6=(@1+(@3*(@4+@5))), @7=((@2-cos(theta))/sq(theta)), @8=(@7*theta_3), @9=(@3*theta_1), @10=(@9*theta_2), @11=(-(@8+@10)), @12=(@7*theta_2), @13=(@9*theta_3), @14=(@12-@13), @15=0, @16=(@8-@10), @17=sq(theta_1), @18=(@1+(@3*(@17+@5))), @19=(@7*theta_1), @20=((@3*theta_2)*theta_3), @21=(-(@19+@20)), @22=(-(@13+@12)), @23=(@19-@20), @24=(@1+(@3*(@4+@17))), @25=2, @26=((@25*theta_2)*y), @27=((@25*theta_3)*z), @28=(@7*z), @29=(@3*((theta_1*y)+(theta_2*x))), @30=(@7*y), @31=(@3*((theta_1*z)+(theta_3*x))), @32=((@25*theta_1)*x), @33=(@7*x), @34=(@3*((theta_2*z)+(theta_3*y))), 
[[@6, @16, @22, (@3*(@26+@27)), (@28-@29), (-(@30+@31))], 
 [@11, @18, @23, (-(@28+@29)), (@3*(@27+@32)), (@33-@34)], 
 [@14, @21, @24, (@30-@31), (-(@33+@34)), (@3*(@26+@32))], 
 [@15, @15, @15, @6, @16, @22], 
 [@15, @15, @15, @11, @18, @23], 
 [@15, @15, @15, @14, @21, @24]])

dot_plot_draw output as "result_test_simplified.png"

In [6]:
dot_plot_draw(UInv_ca)
